In [1]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision
import torchvision.transforms as transforms
import numpy as np

from tqdm import tqdm

from concrete.ml.torch.compile import compile_torch_model

/home/local/ASURITE/nnjungle/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#hyperparams
no_epochs = 10
batch_size = 32
learning_rate = 0.001

In [4]:
#transform the dataset into tensors normalized range [-1, 1]
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))     
        ])

In [5]:
#data sets downloading and reading
train_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform
                                        )

test_dataset = torchvision.datasets.CIFAR10(root='data',
                                        train=False,
                                        download=True,
                                        transform=transform
                                        )

100%|██████████| 170498071/170498071 [00:03<00:00, 47836831.91it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
# The Dataloaders - used to load data into application for use
train_loader = torch.utils.data.DataLoader(train_dataset,
                            batch_size=batch_size,
                            shuffle=True
                            )

test_loader = torch.utils.data.DataLoader(test_dataset,
                                    batch_size=batch_size,
                                    shuffle=False
                                    )

In [7]:
class ConvolutionalNueralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(64*4*4, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = functional.relu(self.conv1(x))
        x = self.pool(x)
        x = functional.relu(self.conv2(x))
        x = self.pool(x)
        x = functional.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [8]:
model = ConvolutionalNueralNet().to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)

In [10]:

for epoch in range(no_epochs):
    #loaded_model
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
    
    print(f'[{epoch+1 }] loss: {running_loss/n_total_steps:.3f}')

    print('Finished Training')
    PATH ='./cnn.pth'
    torch.save(model.state_dict(), PATH)

[1] loss: 1.451
Finished Training
[2] loss: 1.085
Finished Training
[3] loss: 0.932
Finished Training
[4] loss: 0.832
Finished Training
[5] loss: 0.760
Finished Training
[6] loss: 0.703
Finished Training
[7] loss: 0.655
Finished Training
[8] loss: 0.613
Finished Training
[9] loss: 0.572
Finished Training
[10] loss: 0.541
Finished Training


In [12]:
loaded_model = ConvolutionalNueralNet().to(device)
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.eval()

with torch.no_grad():
    n_correct = 0
    n_correct2 = 0
    n_samples = len(test_loader.dataset)

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        #max returns (value, index)
        _, predicted = torch.max(outputs,1)
        n_correct +=(predicted == labels).sum().item()
        
        outputs2 = loaded_model(images)
        _, predicted2 = torch.max(outputs, 1)
        n_correct2 +=(predicted2 == labels).sum().item()

    acc = 100.0 * n_correct/ n_samples
    print(f'Accuracy of the model: {acc} %')

    acc = 100.0 * n_correct2 / n_samples
    print(f'Accuracy of the loaded model: {acc} %')


Accuracy of the model: 73.19 %
Accuracy of the loaded model: 73.19 %


In [ ]:
#Work now with concrete

def test_with_concrete(quantized_module, test_loader, use_sim):
    """Test a neural network that is quantized and compiled with Concrete ML."""

    # Casting the inputs into int64 is recommended
    all_y_pred = np.zeros((len(test_loader)), dtype=np.int64)
    all_targets = np.zeros((len(test_loader)), dtype=np.int64)

    # Iterate over the test batches and accumulate predictions and ground truth labels in a vector
    idx = 0
    for data, target in tqdm(test_loader):
        data = data.numpy()
        target = target.numpy()

        fhe_mode = "simulate" if use_sim else "execute"

        # Quantize the inputs and cast to appropriate data type
        y_pred = quantized_module.forward(data, fhe=fhe_mode)

        endidx = idx + target.shape[0]

        # Accumulate the ground truth labels
        all_targets[idx:endidx] = target

        # Get the predicted class id and accumulate the predictions
        y_pred = np.argmax(y_pred, axis=1)
        all_y_pred[idx:endidx] = y_pred

        # Update the index
        idx += target.shape[0]

    # Compute and report results
    n_correct = np.sum(all_targets == all_y_pred)

    return n_correct / len(test_loader)

In [ ]:
n_bits = 6

q_module = compile_torch_model(model, n_total_steps, rounding_threshold_bits=6, p_error=0.1)

start_time = time.time()
accs = test_with_concrete(
    q_module,
    train_loader,
    use_sim=True,
)
sim_time = time.time() - start_time

print(f"Simulated FHE execution for {n_bits} bit network accuracy: {accs:.2f}%")

In [ ]:
# Generate keys first
t = time.time()
q_module.fhe_circuit.keygen()
print(f"Keygen time: {time.time()-t:.2f}s")

In [ ]:

t = time.time()
accuracy_test = test_with_concrete(
    q_module,
    test_loader,
    use_sim=False,
)
elapsed_time = time.time() - t
time_per_inference = elapsed_time / len(test_loader)
accuracy_percentage = 100 * accuracy_test

print(
    f"Time per inference in FHE: {time_per_inference:.2f} "
    f"with {accuracy_percentage:.2f}% accuracy"
)